In [1]:
from lxml import etree

In [2]:
tree = etree.parse("model.xml")

In [9]:
tree.xpath("//model/classes/class")

[<Element class at 0x7fcd16dfd040>,
 <Element class at 0x7fcd16d63180>,
 <Element class at 0x7fcd04e0ba00>,
 <Element class at 0x7fcd04e32240>,
 <Element class at 0x7fcd04e331c0>,
 <Element class at 0x7fcd04e33280>,
 <Element class at 0x7fcd04e332c0>]

In [12]:
for cls in tree.xpath("//model/classes/class"):
    cls_id = cls.xpath("./@ID")[0]
    for att in cls.xpath("./properties/property"):
        print(att.tag)
        att_id = att.xpath("./@ID")[0]
        print(att_id)

property
person.name
property
person.first_name
property
person.start_date_written
property
person.end_date_written
property
person.gender
property
person.class_affiliation
property
person.alternative_label
property
function.name
property
function.alternative_label
property
place.name
property
place.alternative_label
property
place.type
property
place.latitude
property
place.longitude
property
institution.name
property
institution.alternative_label
property
institution.type
property
institution.start_date_written
property
institution.end_date_written
property
court.name
property
court.alternative_label
property
court.type
property
court.start_date_written
property
court.end_date_written
property
event.name
property
event.alternative_label
property
event.type
property
event.start_date_written
property
event.end_date_written
property
salary.amount
property
salary.currency
property
salary.repetitions
property
salary.repetition_note
property
salary.type
property
salary.start_date_written
p

In [3]:
from jinja2 import Environment, FileSystemLoader

In [4]:
jinja_env = Environment(loader=FileSystemLoader('django_orm_templates/'), autoescape=False)

In [3]:
query_template = jinja_env.get_template("entity.txt").render({"cls": "person", "base_cls": "test", "properties": ["test2", "test3"]})

In [5]:
q2 = jinja_env.get_template("prop_choicefield.txt").render({"name": "test1", "length": 15, "choices": ["test1", "test2"]})

In [8]:
query_template

'@reversion.register(follow=["tempentityclass_ptr"])\nclass person(test):\n    # auto generated from model xml\n    \n    test2\n    \n    test3\n    '

In [7]:
with open("test2.py", "w") as out:
    out.write(q2)

In [6]:
q2

'TEST1_CHOICES = (("test1", "test1"), ("test2", "test2"), )\ntest1 = models.CharField(max_length=15, choices=TEST1_CHOICES, blank=True)'

In [8]:
TEST1_CHOICES = (("test1", "test1"), ("test2", "test2"), )

In [3]:

q3 = jinja_env.get_template("rel_property.txt").render({"name": "data read from", "name_reverse": "read opposite", "subjects": ["person", "places"], "objects": ["events"]})

In [4]:
q3

'data_read_from = Property.objects.create(\n        name="data read from",\n        name_reverse="read opposite",\n    )\ndata_read_from.subj_class.add(ContentType.objects.get(model=person.__name__))\ndata_read_from.subj_class.add(ContentType.objects.get(model=places.__name__))\n\ndata_read_from.obj_class.add(ContentType.objects.get(model=events.__name__))\n'

In [5]:
with open("test3.py", "w") as out:
    out.write(q3)

In [5]:
map_fields = {
    "float": "floatfield",
    "int": "integerfield",
    "shortText": "charfield",
    "longText": "textfield",
    "choiceField": "choicefield"   
}

In [6]:
entities = []
relations = {}
for cls in tree.xpath("//model/classes/class"):
    cls_id = cls.xpath("./@ID")[0]
    properties = []
    for att in cls.xpath("./properties/property"):
        att_id = att.xpath("./@ID")[0]
        if "date_written" in att_id or att_id.split(".")[-1] == "name":
            continue
        print(att_id)
        datatype = att.xpath("./datatypeName/@target")[0]
        vocabref = att.xpath("./datatypeName/@vocabRef")
        print(datatype)
        if datatype == "choiceField" and len(vocabref) > 0:
            lst_choices = tree.xpath(f"//vocab[@ID = '{vocabref[0]}']/values/list/item/text()")
            print(lst_choices)
            properties.append(jinja_env.get_template("prop_choicefield.txt").render({"name": att_id.split(".")[-1], "length": max([len(x) for x in lst_choices]), "choices": lst_choices}))
        else:
            properties.append(jinja_env.get_template(f"prop_{map_fields[datatype]}.txt").render({"name": att_id.split(".")[-1]}))
    print(properties)
    entities.append(jinja_env.get_template("entity.txt").render({"cls": cls_id, "base_cls": "TempEntityClass", "properties": properties}))
    for rel in cls.xpath("./relations/relation"):
        src = rel.xpath("./sourceClass/@target")[0].split(" ")
        trgt = rel.xpath("./targetClass/@target")[0].split(" ")
        name = rel.xpath("./name/text()")[0]
        name_reverse = rel.xpath("./reverseName/text()")[0]
        if name not in relations:
            relations[name] = {
                "name": name,
                "name_reverse": name_reverse,
                "subjects": src,
                "objects": trgt,
                }
        else:
            if relations[name]["name_reverse"] != name_reverse:
                print("You got a mismatch in vocabs names")
            else:
                if src not in relations["name"]["subjects"]:
                    relations["name"]["subjects"].extend(src)
                if trgt not in relations["name"]["objects"]:
                    relations["name"]["objects"].extend(trgt)
rels2 = []
for vals in relations.values():
    rels2.append(jinja_env.get_template("rel_property.txt").render(vals))
models = jinja_env.get_template("models.txt").render({"entities": entities, "relations": rels2})

person.name
shortText
person.first_name
shortText
person.gender
choiceField
['männlich', 'weiblich', 'unbekannt']
person.class_affiliation
choiceField
['Hoher Adel', 'Niederer Adel', 'Klerus', 'Bürgertum', 'Bauerntum', 'Gesinde']
person.alternative_label
longText
['name = models.CharField(max_length=1024, blank=True, null=True)', 'first_name = models.CharField(max_length=1024, blank=True, null=True)', 'GENDER_CHOICES = (("männlich", "männlich"), ("weiblich", "weiblich"), ("unbekannt", "unbekannt"), )\n    gender = models.CharField(max_length=9, choices=GENDER_CHOICES, blank=True)', 'CLASS_AFFILIATION_CHOICES = (("Hoher Adel", "Hoher Adel"), ("Niederer Adel", "Niederer Adel"), ("Klerus", "Klerus"), ("Bürgertum", "Bürgertum"), ("Bauerntum", "Bauerntum"), ("Gesinde", "Gesinde"), )\n    class_affiliation = models.CharField(max_length=13, choices=CLASS_AFFILIATION_CHOICES, blank=True)', 'alternative_label = models.TextField(blank=True, null=True)']
function.name
shortText
function.alternati

In [7]:
with open("test_model2.py", "w") as out:
    out.write(models)

In [31]:
models

'import reversion\nfrom django.contrib.contenttypes.models import ContentType\nfrom django.db import models\nfrom apis_core.apis_entities.models import TempEntityClass\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ndef construct_properties():\n\n    from apis_core.apis_vocabularies.models import TextType\n    from apis_core.apis_metainfo.models import Collection\n    from apis_core.apis_relations.models import Property\n    from apis_highlighter.models import AnnotationProject, Project\n    from django.contrib.auth.models import User\n\n    '